## Part A -  Reactive Behaviours

### Constants 

In [3]:
DEFAULT_SPEED = 0.7
RANDOM_SPEED = 0.4
MAX_DEPTH = 600
MAX_SPEED = 2
HUMAN_DETECTION = 1
BOTTLE_DETECTION = 44
BACKPACK_DETECTION = 27
CUP_DETECTION = 47
ROTATING_TIME = 1
LOVE = "Love"
FEAR = "Fear"
AGGRESSIVE = "Aggressive"
CURIOUS = "Curious"

### TensorRT Code

In [2]:
import tensorrt as trt
from tensorrt_model import TRTModel
from ssd_tensorrt import load_plugins, parse_boxes,TRT_INPUT_NAME, TRT_OUTPUT_NAME
import ctypes
import numpy as np
import random
import cv2
import os
import ctypes
    
mean = 255.0 * np.array([0.5, 0.5, 0.5])
stdev = 255.0 * np.array([0.5, 0.5, 0.5])

def bgr8_to_ssd_input(camera_value):
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1)).astype(np.float32)
    x -= mean[:, None, None]
    x /= stdev[:, None, None]
    return x[None, ...]

def draw_box(bbox, image, rgb):
    cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), rgb, 2)

class ObjectDetector(object):
    
    def __init__(self, engine_path, preprocess_fn=bgr8_to_ssd_input):
        logger = trt.Logger()
        trt.init_libnvinfer_plugins(logger, '')
        load_plugins()
        self.trt_model = TRTModel(engine_path, input_names=[TRT_INPUT_NAME],output_names=[TRT_OUTPUT_NAME, TRT_OUTPUT_NAME + '_1'])
        self.preprocess_fn = preprocess_fn
        
    def execute(self, *inputs):
        trt_outputs = self.trt_model(self.preprocess_fn(*inputs))
        return parse_boxes(trt_outputs)
    
    def __call__(self, *inputs):
        return self.execute(*inputs)

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')


### Main Behaviour Parent class with all its child classes.

In [4]:
class Behaviours:
    """
        Main Parent class for all behaviours. 
    """
    
    def __init__(self):
        self.turn_gain = 0
        self.speed = DEFAULT_SPEED
        self.randomVal = random.uniform(1,1.5)
        self.startTime = cv2.getTickCount()
        self.predefined_targets ={
            1: {
                "name" : "HUMAN",
                "status" : "NOT_VISITED"
            }
        }
    
    def move(self, center, movement):
        """
            Move robot based on the movement.
            
            :param center : tuple containing (x,y) coordinate of the center of the target object.
            :param movement : List containing the direction and turn gain in which the obot has to move.
            :return : None
        """ 
        left_speed = float(self.speed + movement[1] * center[0])
        right_speed = float(self.speed - movement[1] * center[0])
        robot.set_motors(left_speed, right_speed, left_speed, right_speed)
    
    def get_min_depth(self, bbox):
        depth_image = np.copy(camera.depth_image)
        #we only consider the central area of the vision sensor
        depth_image[:int(height * bbox[1]),:]=0
        depth_image[int(height * bbox[3]):,:]=0
        depth_image[:,:int(width * bbox[0])]=0
        depth_image[:,int(width * bbox[2]):]=0

        #For object avoidance, we don't consider the distance that are lower than 100mm or bigger than 1000mm
        depth_image[depth_image<100]=0
        depth_image[depth_image>1000]=0

        #If all of the values in the depth image is 0, the depth[depth!=0] command will fail
        #we set a specific value here to prevent this failure
        depth_image[0,0]=2000

        return(depth_image[depth_image!=0].min())

    def turn_left(self, speed = 0.1):    
        """
            Randomly moves the Robot.
        """
        left_speed = float(speed - 0.4)
        right_speed = float(speed + 0.4)
        robot.set_motors(left_speed, right_speed, left_speed, right_speed)
    
    def move_robot_randomly(self):
        currTime = cv2.getTickCount()
        deltaTime = (currTime - self.startTime) / cv2.getTickFrequency()
      
        if deltaTime >= 0 and deltaTime <= self.randomVal:
            self.turn_left()
        elif deltaTime > 2:
            self.startTime = cv2.getTickCount()
            self.randomVal = random.uniform(1,1.5)
        else:
            robot.forward(0.3)
        return
    
    def set_speed(self,depth):
        return None
    
    def get_target(self, object_detections):
        idx = 0
        target= None
        # Depths of all the detections
        detection_depths = np.array([])
        
        for det in object_detections:
            bbox = det['bbox']
            detection_depths = np.append(detection_depths, self.get_min_depth(bbox))
        
        if not len(object_detections):
            return idx, target
        
        # get the min depth of human bboxs
        index_value = np.where(detection_depths == np.amin(detection_depths))
        idx = index_value[0][0]
        target = object_detections[idx]
        
        if detection_depths[idx] > 800:
            return 0, None
        
        
        camera.behaviour.set_speed(detection_depths[idx])
        return idx, target

In [5]:
class Love (Behaviours):
    """
        Subclass of Behaviours. Implements the Love behaviour of robot. Follows the target 
        and gradually slows down as it goes closer to the target.
    """
    def __init__(self):
        super().__init__()
    
    def move(self, center, movement):
        # calling the super class move() function         
        super().move(center, movement)
        
    def set_speed(self,depth):
        """
            Defines the speed of the Robot based on the depth between the robot and target object, as to display a 
            gradually decrease of speed as it moves closer to target. 
            
            :param depth : The depth between the robot and the target object.
            :return : None
        """
        if depth <= MAX_DEPTH:
            self.speed = 0
        elif (depth <=  MAX_DEPTH + 50):
            self.speed = DEFAULT_SPEED - DEFAULT_SPEED * 0.85 
        elif (depth <=  MAX_DEPTH + 100):
            self.speed = DEFAULT_SPEED - DEFAULT_SPEED * 0.75
        elif (depth <= MAX_DEPTH + 150):
            self.speed = DEFAULT_SPEED - DEFAULT_SPEED * 0.55  
        elif (depth <= MAX_DEPTH + 200):
            self.speed = DEFAULT_SPEED - DEFAULT_SPEED * 0.35
        else:
            self.speed = DEFAULT_SPEED
        return None

In [6]:
class Aggressive (Behaviours):
    """
        Implements the Aggressive behaviour of the robot. Move towards target at Max speed and stop immediately,
        as it reaches the target.
    """
    def __init__(self):
        super().__init__()
        #  Pre-define targets, the robot will show aggressive behaviour with these targets.
        self.predefined_targets ={
            BACKPACK_DETECTION: {
                "name" : "HUMAN",
                "status" : "NOT_VISITED"
            }
        }
    def move(self, center, movement):
        """
            Calling the parent/super class move function.
        """
        new_movement = movement[0], movement[1]*5
        super().move(center, new_movement)
        
    def set_speed(self, depth):
        """
            Defines the speed of the robot based on the depth of robot from its target. Makes the speed MAX, 
            when the robot follows the target, and 0 as it becomes lesser than or equal to the Maximum depth.
            Displaying aggressive behaviour of the robot.
            
            :param depth : The depth between the robot and the target object.
            :return : None
        """
        if depth <= MAX_DEPTH:
            self.speed = 0
        else:
            self.speed = MAX_SPEED

In [35]:
class Fear (Behaviours):
    """
        Implements the Fear behaviour of the robot. Turns 180° as it is inside target's range, ie. MAX_DEPTH.
    """
    
    def __init__(self):
        super().__init__()
        self.turn_flag = False
        self.isRobotTurning = False
        self.startTime = cv2.getTickCount()
        #  Pre-define targets, the robot will show fear behaviour with these targets.
        self.predefined_targets ={
            44: {
                "name" : "BOTTLE",
                "status" : "NOT_VISITED"
            }
        }
    
    def set_speed(self, depth):
        """
            Defines the speed of the robot based on the depth of robot from its target. Makes the speed MAX, 
            when the robot follows the target, and 0 as it becomes lesser than or equal to the Maximum depth.
            Displaying aggressive behaviour of the robot.
            
            :param depth : The depth between the robot and the target object.
            :return : None
        """
        if depth <= MAX_DEPTH:
            self.turn_flag = True
        elif self.isRobotTurning == False:
            self.turn_flag = False 
            
    def move(self, center, movement):
        """
            Rotates the robot 180°, as it's get inside the target's range, then moves the robot randomly.
        """
        # Testing: Rotate the robot 180 degree.
        if self.turn_flag:
            self.isRobotTurning = True
            currTime = cv2.getTickCount()
            deltaTime = (currTime - self.startTime) / cv2.getTickFrequency()
            if deltaTime <= 6:
                robot.left(MAX_SPEED)
            else: 
                robot.stop()
                self.isRobotTurning  = False
        else:
            self.isRobotTurning  = False
            self.startTime = cv2.getTickCount()
            super().move_robot_randomly()            

In [8]:
class Curious (Behaviours):
    """
        Child class of Behaviours class. Implements the curious behaviour of the robot class.
    """
    
    def __init__(self):
        super().__init__()
        self.target_reached = False
        # Pre-defined target objects dictionary, for displaying curious attribute of the robot.
        self.predefined_targets ={
            44: {
                "name" : "BOTTLE",
                "status" : "NOT_VISITED"
            },
            1: {
                "name" : "BACKPACK",
                "status" : "NOT_VISITED"
            }
        }
    
    def set_speed(self, depth):    
        """
        Defines the speed of the robot. And status of the target object becomes "VISITED", once depth reaches it
        maximum value.
        
        :param depth : The depth between the robot and the target object.
        :return : None
        """
        if depth <= MAX_DEPTH:
            self.speed = 0
            self.target_reached = True
            for i in self.predefined_targets:
                if self.predefined_targets[i]["status"] == "VISITING":
                    self.predefined_targets[i]["status"] = "VISITED"   
                    break
        else:
            self.speed = DEFAULT_SPEED
        return None
    
    def get_target(self, object_detections):
        """
            Returns the element that has a status "VISITING", if object_detections contains no such predefined_target, it returns 
            a random element as target with its index from the predefined_targets, that object_detections contains.
            
            :param object_detections : Dictionary containing all the predefined_target objects that are present 
                                      in the camera frame.
            :return target : the element from the object_detections dictionary that has a status "VISTING".
            :return idx    : the index number of the target element.
                    
        """
        idx = 0
        target= None
                
        for det in self.predefined_targets:
            if self.predefined_targets[det]["status"] == "VISITING":
                idx , target = super().get_target([i for i in object_detections if i['label'] == det])
                if target is None:
                    break
                    
                idx = np.where(object_detections == target)[0]
                return idx, target
                
        if target is None:
            random_index = random.randint(0,len(object_detections) - 1)
            target = object_detections[random_index]
            idx = random_index
            self.predefined_targets[target['label']]["status"] = "VISITING"

        return idx, target
    

    def move(self, center, movement):
        """
            Stop robot as it gets in the target's range for 2 sec and then rotates it 180°, else 
            follows the target untill it reach. 
        """
        if self.target_reached:
            robot.stop()
            time.sleep(2)
            # Rotating the robot 180 degrees             
            robot.left(DEFAULT_SPEED)
            time.sleep(ROTATING_TIME)
            self.speed = DEFAULT_SPEED
            self.target_reached = False
            
        super().move(center, movement)
        

### Initialize the camera instance for the Intel realsense sensor D435i

In [9]:
#use traitlets and widgets to display the image in Jupyter Notebook
import traitlets
from traitlets.config.configurable import SingletonConfigurable

#use opencv to covert the depth image to RGB image for displaying purpose
import cv2
import numpy as np

#using realsense to capture the color and depth image
import pyrealsense2 as rs

#multi-threading is used to capture the image in real time performance
import threading

class Camera(SingletonConfigurable):
    
    #this changing of this value will be captured by traitlets
    color_value = traitlets.Any()
    
    def __init__(self):
        super(Camera, self).__init__()
        
        #configure the color and depth sensor
        self.pipeline = rs.pipeline()
        self.configuration = rs.config()  
        
        #set resolution for the color camera
        self.color_width = 640
        self.color_height = 480
        self.color_fps = 30
        self.configuration.enable_stream(rs.stream.color, self.color_width, self.color_height, rs.format.bgr8, self.color_fps)

        #set resolution for the depth camera
        self.depth_width = 640
        self.depth_height = 480
        self.depth_fps = 30
        self.configuration.enable_stream(rs.stream.depth, self.depth_width, self.depth_height, rs.format.z16, self.depth_fps)

        #flag to control the thread
        self.thread_runnning_flag = False
        
        #start the RGBD sensor
        self.pipeline.start(self.configuration)
        self.pipeline_started = True
        frames = self.pipeline.wait_for_frames()

        #start capture the first color image
        color_frame = frames.get_color_frame()   
        image = np.asanyarray(color_frame.get_data())
        self.color_value = image

        #start capture the first depth image
        depth_frame = frames.get_depth_frame()           
        depth_image = np.asanyarray(depth_frame.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        self.depth_value = depth_colormap   

        # Intialising Behaviour class.
        self.behaviour = Behaviours()
        self.target_detected = False
        self.target_center = ()
        
        
    def _capture_frames(self):
        while(self.thread_runnning_flag==True): #continue until the thread_runnning_flag is set to be False
            frames = self.pipeline.wait_for_frames() #receive data from RGBD sensor
            
            color_frame = frames.get_color_frame() #get the color image
            image = np.asanyarray(color_frame.get_data()) #convert color image to numpy array
            self.color_value = image #assign the numpy array image to the color_value variable 

            depth_frame = frames.get_depth_frame() #get the depth image           
            depth_image = np.asanyarray(depth_frame.get_data()) #convert depth data to numpy array
            #conver depth data to BGR image for displaying purpose
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
            self.depth_value = depth_colormap #assign the color BGR image to the depth value
            cv2.putText(depth_colormap, str(depth_image[240,320]), (320,240), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)            
            self.depth_image = depth_image 
                    
                
    def start(self): #start the data capture thread
        if self.thread_runnning_flag == False: #only process if no thread is running yet
            self.thread_runnning_flag=True #flag to control the operation of the _capture_frames function
            self.thread = threading.Thread(target=self._capture_frames) #link thread with the function
            self.thread.start() #start the thread

    def stop(self): #stop the data capture thread
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False #exit the while loop in the _capture_frames
            self.thread.join() #wait the exiting of the thread       

def bgr8_to_jpeg(value):#convert numpy array to jpeg coded data for displaying 
    return bytes(cv2.imencode('.jpg',value)[1])

#create a camera object
camera = Camera.instance()
camera.start() # start capturing the data


### Perform object detection and display the results on widgets

In [10]:
import ipywidgets.widgets as widgets
from IPython.display import display, HTML

width = 640
height = 480

width_left_edge = 190
width_right_edge = 450

image_widget = widgets.Image(format='jpeg', width=width, height=height)

# Creating Drop down menu for target objects.
dropDownMenu = widgets.Dropdown(
    options=[LOVE, FEAR, AGGRESSIVE, CURIOUS],
    value=LOVE,
    description='Behaviour:',
)

# Default behaviour is set to LOVE
camera.behaviour = Love()

def onItemSelected(item):
    if item['type'] == 'change' and item['name'] == 'value':
        selected_behaviour = item['new']
        if selected_behaviour == LOVE:
            camera.behaviour = Love()
        elif selected_behaviour == FEAR:
            camera.behaviour = Fear()
        elif selected_behaviour == AGGRESSIVE:
            camera.behaviour = Aggressive()
        elif selected_behaviour == CURIOUS:
            camera.behaviour = Curious()
        else:
            print("Something went wrong!")

# Attaching a listener for drop down menu
dropDownMenu.observe(onItemSelected)


### MAIN ROBOT PROCESSING CODE : Perform object detection and controls the robot

In [37]:
import time
from RobotClass import Robot

# Initializes the Robot class
robot = Robot()

# https://github.com/NVIDIA-AI-IOT/jetbot/blob/master/notebooks/object_following/live_demo.ipynb
def get_center(bbox):
    """Computes the center x, y coordinates of the object"""
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)

def detect_section(coord):
    """
        Detects what section the object is in and calculates the turn gain.
        turn gain is between 0 and 1
    """
    direction = "CENTER"
    turn_gain = 0
    center_x = coord[0]
    
    if(center_x < width_left_edge):
        turn_gain = (width_left_edge - center_x) / width_left_edge
        direction= "LEFT"
    elif(center_x > width_right_edge):
        turn_gain = (center_x - width_right_edge) / width_left_edge
        direction= "RIGHT"
   
    return direction, turn_gain

def processing(change):
    """
        Runs every Frame on a different thread (other than the main thread)
    """
    image = change['new']
    
    imgsized= cv2.resize(image,(300,300))
    
    # compute all detected objects
    detections = model(imgsized)
    
    # https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt
    
    # Will return us a object_detections dictionary of the objects that are present in predefined_target 
    # from every objects inside the camera frame. So, we only detect those objects that are present inside the
    # object_detections dictionary.
    predefined_keys = [d for d in camera.behaviour.predefined_targets if camera.behaviour.predefined_targets[d]['status'] != "VISITED"]
    object_detections = [d for d in detections[0] if d['label'] in predefined_keys]
        
    if len(object_detections) == 0:
        camera.target_detected = False
        camera.target_center = None
    else:
        
        idx, target = camera.behaviour.get_target(object_detections)
        
        if target is not None:
            
            bbox = target['bbox']

            # Display rectangle on the camera widget        
            draw_box(bbox, image, (0,0,255))
            # When the robot is too near to the target object
            center = get_center(bbox)

            # set camera speed based on depth   
            camera.target_detected = True
            camera.target_center = center
        else:
            camera.target_detected = False
            camera.target_center = (0,0)
            
    # Move towards target by calling the move() function based on the type of behaviour object. 
    if(camera.target_detected):
        center = camera.target_center
        movement = detect_section(center)
        camera.behaviour.move(center, movement)        
    else:
        # Moves robot randomly         
        camera.behaviour.move_robot_randomly()
                              
                          
    image_widget.value = bgr8_to_jpeg(image)
    
#the camera.observe function will monitor the color_value variable. If this value changes, the excecute function will be excuted.
camera.observe(processing, names='color_value')

### Displaying the Camera Image widget

In [36]:
display(widgets.VBox([
    dropDownMenu,
    image_widget]))

### Distaches the observe listener from the camera object and stops the robot.

In [38]:
camera.unobserve_all()
camera.stop()
camera.start()
time.sleep(1.0)
robot.stop()